# Laboratorio 1 Phishing

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/dataset_pishing.csv')

df.head()

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


In [5]:
status_counts = df['status'].value_counts()
print("Cantidad de observaciones por etiqueta:")
print(status_counts)
print(f"\nTotal de observaciones: {len(df)}")

print("\nPorcentajes:")
print(df['status'].value_counts(normalize=True) * 100)

Cantidad de observaciones por etiqueta:
status
legitimate    5715
phishing      5715
Name: count, dtype: int64

Total de observaciones: 11430

Porcentajes:
status
legitimate    50.0
phishing      50.0
Name: proportion, dtype: float64


EL DATASET SE ENCUENTRA BALANCEADO